In [1]:
import pandas as pd
#import dnnlib
#import dnnlib.tflib as tflib
import tfutil
import dataset
import misc
import numpy as np
import config 
import tensorflow as tf
import os

import pickle
#tflib.init_tf()
#path = './results/000-pgan-cifar10_wm_mullabel-cond-preset-v2-1gpu-fp32'
#path = './pre_model'
#G, D, Gs   = misc.load_pkl(path+'/network-snapshot-002705.pkl')
#network_pkl = misc.locate_network_pkl(0, 3747)
#print(network_pkl)

In [2]:
# resume_run_id = 4
# resume_snapshot = 12000
path = 'model/'
tfutil.init_tf(config.tf_config)
with tf.device('/gpu:0'):
#     if resume_run_id is not None:
        #network_pkl = misc.locate_network_pkl(resume_run_id, resume_snapshot)
    network_pkl = misc.locate_network_pkl(path+'progan-mullabel3000-003751.pkl')
    print('Loading networks from "%s"...' % network_pkl)
    G, D, Gs = misc.load_pkl(network_pkl)
            
#print(type(Gs))
#Gs.print_layers()

W0721 06:26:53.153005 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:56: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0721 06:26:53.154122 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:57: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0721 06:26:53.170404 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:65: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0721 06:26:53.170998 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:72: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0721 06:26:54.676268 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_

Loading networks from "model/progan-mullabel3000-003751.pkl"...
---226tfutil--- <module '_tfutil_network_import_module_0'> ----  G_paper ------
---226tfutil--- <module '_tfutil_network_import_module_1'> ----  D_paper ------
---226tfutil--- <module '_tfutil_network_import_module_2'> ----  G_paper ------
<class 'tfutil.Network'>

Gs                          Params      OutputShape             WeightShape             
---                         ---         ---                     ---                     
latents_in                  -           (?, 512)                -                       
labels_in                   -           (?, 10)                 -                       
lod                         -           ()                      -                       
4x4/PixelNorm               -           (?, 522)                -                       
4x4/Dense                   4276736     (512,)                  (522, 8192)             
4x4/Conv                    2359808     (512,) 

In [3]:
#dataset_args = EasyDict(tfrecord_dir='cifar10_wm_mullabel')
training_set = dataset.load_dataset(data_dir=config.data_dir, verbose=True, **config.dataset)
print('load finish')

W0721 06:27:26.339923 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:72: The name tf.python_io.TFRecordOptions is deprecated. Please use tf.io.TFRecordOptions instead.

W0721 06:27:26.341089 140021333210944 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:72: The name tf.python_io.TFRecordCompressionType is deprecated. Please use tf.compat.v1.python_io.TFRecordCompressionType instead.

W0721 06:27:26.342031 140021333210944 deprecation.py:323] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:73: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0721 06:27:26.409500 140021333210944 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals

Streaming data using dataset.TFRecordDataset...
---232--- dataset.TFRecordDataset ------
------214----- ['dataset', 'TFRecordDataset']
------217----- dataset
-----221---- <module 'dataset' from '/tf/notebooks/progressive_growing_of_gans-master/dataset.py'>
---226tfutil--- <module 'dataset' from '/tf/notebooks/progressive_growing_of_gans-master/dataset.py'> ----  TFRecordDataset ------
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 10
load finish


In [4]:
def setup_snapshot_image_grid(G, training_set,
    size    = '1080p',      # '1080p' = to be viewed on 1080p display, '4k' = to be viewed on 4k display.
    layout  = 'random'):    # 'random' = grid contents are selected randomly, 'row_per_class' = each row corresponds to one class label.

    # Select size.
    gw = 1; gh = 1
    if size == '1080p':
        gw = np.clip(1920 // G.output_shape[3], 3, 32)
        gh = np.clip(1080 // G.output_shape[2], 2, 32)
    if size == '4k':
        gw = np.clip(3840 // G.output_shape[3], 7, 32)
        gh = np.clip(2160 // G.output_shape[2], 4, 32)

    # Fill in reals and labels.
    reals = np.zeros([gw * gh] + training_set.shape, dtype=training_set.dtype)
    labels = np.zeros([gw * gh, training_set.label_size], dtype=training_set.label_dtype)
    for idx in range(gw * gh):
        x = idx % gw; y = idx // gw
        while True:
            real, label = training_set.get_minibatch_np(1)
            if layout == 'row_per_class' and training_set.label_size > 0:
                if label[0, y % training_set.label_size] == 0.0:
                    continue
            reals[idx] = real[0]
            labels[idx] = label[0]
            break

    # Generate latents.
    latents = misc.random_latents(gw * gh, G)
    return (gw, gh), reals, labels, latents


In [5]:
grid_size, grid_reals, grid_labels, grid_latents = setup_snapshot_image_grid(G, training_set, **config.grid)
print(grid_size)
print(len(grid_latents))
print(len(grid_latents[0]))

In [8]:
for i in range(1024):
    k = 0
    for j in grid_labels[i]:
        if(j == 1 ):
            k = k + 1
    if k >1:
        print(grid_labels[i])
        break

[1. 0. 0. 0. 1. 1. 0. 0. 0. 1.]


In [9]:
from matplotlib import pyplot as plt
for i in range(len(grid_labels)) :
    #break
    #grid_labels[i][np.where(grid_labels[i] == 1)]
    #print(grid_labels[i])
   # print(np.where(grid_labels[i] == 1)[0][0])
    label = np.where(grid_labels[i] == 1)[0][0]
    
    k=0 
    for j in grid_labels[i]:
        if(j == 1 ):
            k = k + 1
    if k >1:
        label = 11
    grid_fakes = Gs.run(np.matrix(grid_latents[i]),np.matrix(grid_labels[i]), is_validation=True, randomize_noise=True) #这部分需要重新考虑
    misc.save_image_grid(grid_fakes, os.path.join(path + '/progan-mullabel3000-003751/' ,'%dfakes%d.png'%(label,i)), drange=[-1,1], grid_size=[1,1])

In [10]:
print('hihi/'+'byebye')

hihi/byebye
